In [612]:
import pandas as pd
import numpy as np
import collections
import operator

In [613]:
## Here goes the implementation for predicing haplogroups

In [614]:
df_intermediate = pd.read_csv("Hg_Prediction_tables/Intermediates.txt", header=None)
intermediates = df_intermediate[0].values

In [615]:
sample_name = "Output_example_datasets/Ancient_DNA_WGS/WC1/WC1.out"
df_haplogroup = pd.read_csv(sample_name, sep="\t")
#df_haplogroup = pd.read_csv("Output_example_datasets/Modern_DNA_WGS/HG00634/HG00634.out", sep="\t")

In [616]:
df_haplogroup = df_haplogroup.sort_values(by=['haplogroup'])

### Derive State

#### Removes the intermediate haplogroups temporally

In [617]:
df_derived = df_haplogroup[df_haplogroup["state"] == "D"]

In [618]:
## instance with only D state
df_tmp = df_derived
for hg in intermediates:    
    df_tmp = df_tmp.drop(df_tmp[df_tmp.haplogroup == hg].index)
hg = df_tmp["haplogroup"].values

#### Choose the haplogroup based on the highest count in the first letter (E.x. J2a21 = J)

In [619]:
list_hg = []
for i in hg:
    list_hg.append(i)
collections_hg = collections.Counter(list_hg)

In [620]:
init_hg = max(collections_hg.items(), key=operator.itemgetter(1))[0]
max_init_hg = init_hg
init_hg = init_hg[0]

In [621]:
tmp_init_hg = init_hg+"_int.txt"
hg_intermediate_file = "Hg_Prediction_tables/"+tmp_init_hg
df_intermediate = pd.read_csv(hg_intermediate_file, header=None, sep="\t")

### QC.1 
Now you have the putative haplogroup. Open the file with the haplogroup intermediate and show the intermediate haplogroups again and compare if there are some where does not match with the correct state and store it a error. Give the estimate of the  correct_count/total  

$$QC1 = \frac{correct state } {total}$$

where 
###### correct state is the total number of correct haplogroup state found in the intermediate haplogroup file
###### total of all intermediate haplogroup found in the intermediate haplogroup file

In [622]:
total = 0
correct_state = 0
for i in df_intermediate.values:           
    tmp = df_haplogroup.loc[df_haplogroup["haplogroup"] == i[0]]    
    if not tmp.empty:               
        if "/" in i[1]:
            #i = i[1].split("/")[-1] 
            correct_state += len(tmp) 
            total += len(tmp)                           
        else:        
            correct_state += np.sum(i[1] == tmp["state"])                                                
            total += len(tmp)           
qc_one = round((correct_state / total),3)

Removes all haplogroup but the main one
Check if the preffix of all main haplogroup with D state by allowing one haplogroup that does not match to the main haplogroup until you get back to the origin. 
If mismatch > 1 skip choose as main haplogroup the following in the largest length and continue. In case haplogroup contains ~ symbol at the end ignore it temporally for preffix comparison but if this is the main haplogroup store the special character at the end. 

In [623]:
### Create a suffix trie" each branch is a possible path to form the haplogroup
### Run the algorithm again looking for more than two mismatches 
### Then, run the QC scores for each putative_hg and choose the one with the highsest overall-score
"""
list_main_hg = []
for i in hg:    
    if i.startswith(init_hg):
        list_main_hg.append(i.replace("~",""))

list_main_hg = set(list_main_hg)
list_main_hg = sorted(list(set(list_main_hg)), reverse=False)
print(list_main_hg)
if init_hg in list_main_hg:
    putative_hg = ""
    max_mismatch = 2    
    for i in range(len(list_main_hg)-1):        
        mismatch = 0    
        for j in range(i,len(list_main_hg)):                
            current_hg = list_main_hg[i]                        
            next_hg    = list_main_hg[j]
            current_hg = current_hg.replace("~","")
            next_hg    = next_hg.replace("~","")        
            if not next_hg in current_hg:
                mismatch += 1    
            if mismatch >= max_mismatch:
                break
        if mismatch < max_mismatch:
            putative_hg = list_main_hg[i]        
            break
else:
    putative_hg = max_init_hg
putative_hg
"""

'\nlist_main_hg = []\nfor i in hg:    \n    if i.startswith(init_hg):\n        list_main_hg.append(i.replace("~",""))\n\nlist_main_hg = set(list_main_hg)\nlist_main_hg = sorted(list(set(list_main_hg)), reverse=False)\nprint(list_main_hg)\nif init_hg in list_main_hg:\n    putative_hg = ""\n    max_mismatch = 2    \n    for i in range(len(list_main_hg)-1):        \n        mismatch = 0    \n        for j in range(i,len(list_main_hg)):                \n            current_hg = list_main_hg[i]                        \n            next_hg    = list_main_hg[j]\n            current_hg = current_hg.replace("~","")\n            next_hg    = next_hg.replace("~","")        \n            if not next_hg in current_hg:\n                mismatch += 1    \n            if mismatch >= max_mismatch:\n                break\n        if mismatch < max_mismatch:\n            putative_hg = list_main_hg[i]        \n            break\nelse:\n    putative_hg = max_init_hg\nputative_hg\n'

In [624]:
"""
list_main_hg = []
for i in hg:    
    if i.startswith(init_hg):
        list_main_hg.append(i.replace("~",""))
list_main_hg = sorted(list(set(list_main_hg)), reverse=False)
if init_hg in list_main_hg:
    putative_hg = ""
    max_mismatch = 2    
    for i in range(len(list_main_hg)-1):        
        mismatch = 0    
        for j in range(i,len(list_main_hg)):                
            current_hg = list_main_hg[i]            
            next_hg    = list_main_hg[j]
            current_hg = current_hg.replace("~","")
            next_hg    = next_hg.replace("~","")        
            if not next_hg in current_hg:
                mismatch += 1    
            if mismatch >= max_mismatch:
                break
        if mismatch < max_mismatch:
            putative_hg = list_main_hg[i]        
            break
else:
    putative_hg = max_init_hg
putative_hg
"""

'\nlist_main_hg = []\nfor i in hg:    \n    if i.startswith(init_hg):\n        list_main_hg.append(i.replace("~",""))\nlist_main_hg = sorted(list(set(list_main_hg)), reverse=False)\nif init_hg in list_main_hg:\n    putative_hg = ""\n    max_mismatch = 2    \n    for i in range(len(list_main_hg)-1):        \n        mismatch = 0    \n        for j in range(i,len(list_main_hg)):                \n            current_hg = list_main_hg[i]            \n            next_hg    = list_main_hg[j]\n            current_hg = current_hg.replace("~","")\n            next_hg    = next_hg.replace("~","")        \n            if not next_hg in current_hg:\n                mismatch += 1    \n            if mismatch >= max_mismatch:\n                break\n        if mismatch < max_mismatch:\n            putative_hg = list_main_hg[i]        \n            break\nelse:\n    putative_hg = max_init_hg\nputative_hg\n'

### Ancestral and Derive State

### QC.2

Check if the same name of the main haplogroup appears as an Ancestral State and save the number of count and calculate QC2:

$$QC2 = \frac{Dhg-Ahg} {total}$$
where 
###### Dhg is the total number of derived main haplogroup 
###### Ahg is the total number of ancestral with same name of main haplogroup
###### total of all derived and ancestral haplogroup with the same name

In [625]:
list_main_hg = []
for i in hg:    
    if i.startswith(init_hg):
        list_main_hg.append(i.replace("~",""))
list_main_hg = set(list_main_hg)
list_main_hg = sorted(list(set(list_main_hg)), reverse=False)
print(list_main_hg)
list_putative_hg = []
#list_putative_hg = ['G', 'G1a2a1b', 'G1b', 'G2', 'G2a', 'G2a1a1a1a1a1', 'G2a1a2a1a', 'G2a2', 'G2a2a1a1a', 'G2a2a1a2a1b', 'G2a2a1a2a1b1', 'G2a2a2a', 'G2a2b', 'G2a2b2a1a1a1', 'G2a2b2a1a1a1a3', 'G2a2b2a1a1a2a2', 'G2a2b2a1a1a4', 'G2a2b2a1a1b1a1a2a1a3', 'G2a2b2a1a1b1a1e1a2', 'G2a2b2a1a1c1a1a1a', 'G2a2b2a1a1c1b', 'G2a2b2a1a1c2b', 'G2a2b2a1a1c3', 'G2a2b2b1a1c']
qc_two = 0
qc_tmp = 0
for putative_hg in list_main_hg:
    total_qctwo = len(df_haplogroup.loc[df_haplogroup["haplogroup"] == putative_hg])
    Ahg = np.sum("A" == df_haplogroup.loc[df_haplogroup["haplogroup"] == putative_hg]["state"])
    qc_tmp = round((total_qctwo-Ahg)/total_qctwo,3)
    #print(qc_tmp)
    if qc_tmp > 0.70:
        list_putative_hg.append(putative_hg)
        #print(putative_hg)
        qc_two = qc_tmp
        #print("{}: {}".format(putative_hg, qc_two))
    else:
        print("{}: {}".format(putative_hg, qc_tmp))
if len(list_putative_hg) > 0:
    putative_hg = max(list_putative_hg)
print("--------------")
print("{}: {}".format(putative_hg, qc_two))

['G', 'G1', 'G2', 'G2b', 'G2b2', 'G2b2a']
G1: 0.0
G2b2a: 0.25
--------------
G2b2: 1.0


### QC.3
Show both Ancestral and Derived states from the main haplogroup and check the preffix of each haplogroup that is before the main one until you reach the root. If there are some Ancestral states which follows the pattern/preffix from the main haplogroup keep the count of how many of these appear and this will give the total count. Substract the ones found from the corrected and this will give the QC.3 score. 

In [626]:
list_main_hg_all = []
list_hg_all = df_haplogroup["haplogroup"].values
for i in list_hg_all:    
    if i.startswith(init_hg):
        list_main_hg_all.append(i)
list_main_hg_all = sorted(list(set(list_main_hg_all)), reverse=True)
df_main_hg_all = df_haplogroup.loc[df_haplogroup["haplogroup"].isin(list_main_hg_all)]
df_main_hg_all = df_main_hg_all[["haplogroup","state","marker_name"]]
df_main_hg_all = df_main_hg_all.sort_values(by="haplogroup", ascending=False).values

In [627]:
a_match = 0
total_match = 0
for putative_hg in list_putative_hg:
    for i in df_main_hg_all:
        tmp_hg = i[0].replace("~","")
        if tmp_hg in putative_hg:
            total_match +=1        
            if i[1] == "A":
                a_match += 1
    qc_three = round((total_match - a_match) / total_match,3)
    print("{}: {}".format(putative_hg, qc_three))

G: 1.0
G2: 0.997
G2b: 0.997
G2b2: 0.996


### Final output
1) show the haplogroup name 
    if main haplogroup is a preffix from a higher resolution Ancestral state haplogroup report this haplogroup and marker name (Ex. HG: J21a Marker: L123)
    
2) Not J21a-L123

Could be that that there are more than one contain as a preffix from an ancestral state haplogroup. Should report all of them only if there are different haplogroup name with the resolution
**Check this in more detail later on... 

In [628]:
df_putative_ancestral_hg = pd.DataFrame()
for i in df_main_hg_all:    
    if i[0] == putative_hg:
        break
    if putative_hg in i[0]:     
        tmp = pd.DataFrame([i], columns=["haplogroup","state","marker"])        
        df_putative_ancestral_hg = df_putative_ancestral_hg.append(tmp)        

putative_ancestral_hg = []        
if not df_putative_ancestral_hg.empty:
    df_putative_ancestral_hg = df_putative_ancestral_hg.sort_values(by="haplogroup")
    df_putative_ancestral_hg = df_putative_ancestral_hg
    df_putative_ancestral_hg.index = range(len(df_putative_ancestral_hg))    
    
    for i in range(len(df_putative_ancestral_hg)):        
        row = df_putative_ancestral_hg.iloc[i]
        if putative_ancestral_hg == []:        
            putative_ancestral_hg.append(row.values)                    
        else:        
            if putative_ancestral_hg[-1][0] not in row[0]:
                putative_ancestral_hg.append(row.values)                                  
    putative_ancestral_hg = np.array(putative_ancestral_hg)

In [629]:
out_name = sample_name.split("/")[-1]
out_name = out_name.split(".")[0]
for i in df_derived.loc[df_derived["haplogroup"] == putative_hg].values:
    out_hg = ("{}-{}".format(i[3],i[2]))
    break

header = "Sample_name\tHg\tHg_marker\tQC-score\tQC-1\tQC-2\tQC-3"

out_marker = out_hg
if len(putative_ancestral_hg) > 0:
    out_marker += "*("
    for pahg in putative_ancestral_hg:        
        out_marker += pahg[2]+","
    out_marker += ")"
output = "{}\t{}\t{}\t{}\t{}\t{}\t{}".format(out_name,out_hg,out_marker,round((qc_one*qc_two*qc_three),3),qc_one,qc_two,qc_three)
print(header)
print(output)

Sample_name	Hg	Hg_marker	QC-score	QC-1	QC-2	QC-3
WC1	G2b2-FGC3145	G2b2-FGC3145*(Z8024,Z37344,)	0.996	1.0	1.0	0.996
